# Strucutred LLM Outputs
---

### Overview

The project explores the use and benefits of structured LLM outputs. 

## Import libraries

In [ ]:
from openai import OpenAI
import pandas as pd
import os
from pydantic import BaseModel, Field
from IPython.display import Markdown
from typing import Any, Dict, List, Sequence, Union
from dotenv import load_dotenv

## Initialize Client


In [ ]:
load_dotenv() 
API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=API_KEY)

MODEL = "gpt-4o-mini"

## Build MCP Server

The following code implements an interactive chatbot that connects to Claude, lets the model decide when to call Python tools (like searching arXiv or extracting paper info), executes those tools, and feeds the results back into the conversation — until the assistant provides a final, user-facing answer.

### Chatbot Code

The chabot handles queries on a case by case basis. No memory is persistent across queries. 

In [23]:
load_dotenv() 
API_KEY = os.getenv("ANTHROPIC_API_KEY")
client = anthropic.Anthropic(api_key=API_KEY)

In [ ]:
def process_query(query: str) -> None:
    """
    Process a single user query by interacting with the Anthropic LLM
    and handling any tool calls requested by the model.

    This function sends the query to Claude, monitors the response, and:
      - Prints assistant text outputs directly.
      - Executes tool calls if the model requests one.
      - Passes tool results back to the model for further processing.
      - Loops until the model produces a final text-only response.

    Args:
        query (str): The natural language input from the user.

    Side Effects:
        - Prints assistant replies and tool calls to stdout.
    """
    messages = [{'role': 'user', 'content': query}]

    response = client.messages.create(
        max_tokens=2024,
        model='claude-3-7-sonnet-20250219',
        tools=tools,       # assumes `tools` is defined globally
        messages=messages
    )

    active = True
    while active:
        assistant_content = []

        for content in response.content:
            # Case A: Assistant replies with plain text
            if content.type == 'text':
                print(content.text)
                assistant_content.append(content)

                if len(response.content) == 1:
                    active = False

            # Case B: Assistant requests a tool
            elif content.type == 'tool_use':
                assistant_content.append(content)
                messages.append({'role': 'assistant', 'content': assistant_content})

                tool_id = content.id
                tool_args = content.input
                tool_name = content.name

                print(f"Calling tool {tool_name} with args {tool_args}")

                # Execute tool and add result back into conversation
                result = execute_tool(tool_name, tool_args)
                messages.append({
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_id,
                            "content": result
                        }
                    ]
                })

                # Ask model for next step using tool result
                response = client.messages.create(
                    max_tokens=2024,
                    model='claude-3-7-sonnet-20250219',
                    tools=tools,
                    messages=messages
                )

                if len(response.content) == 1 and response.content[0].type == "text":
                    print(response.content[0].text)
                    active = False


def chat_loop() -> None:
    """
    Start an interactive terminal chat loop with the chatbot.

    Users can type queries, which are passed to `process_query` for handling.
    Typing 'quit' ends the session.

    Side Effects:
        - Continuously prompts for user input.
        - Prints assistant replies and tool call logs.
    """
    print("Type your queries or 'quit' to exit.")

    while True:
        try:
            query = input("\nQuery: ").strip()

            if query.lower() == 'quit':
                break

            process_query(query)
            print("\n")

        except Exception as e:
            print(f"\nError: {str(e)}")

## Build MCP Client

The following code takes the functions `process_query` and `chat_loop` and wraps them in a MCP_Chatbot class. This enables the chatbot to communicate with the MCP server.


In [ ]:
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client

The MCP Client library brings in the following classes:

* **`ClientSession`:** manages the connection and sends requests.

* **`StdioServerParameters`:** defines how to launch the server.

* **`stdio_client`:** helper to create a stdio transport.